## Analyze A/B Test Results


## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.


<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [ ]:
df = pd.read_csv('ab_data.csv')
df.head()

In [ ]:
df['timestamp'].min()

In [ ]:
df['timestamp'].max()

b. Use the cell below to find the number of rows in the dataset.

In [ ]:
df.shape

c. The number of unique users in the dataset.

In [ ]:
df['user_id'].nunique()

d. The proportion of users converted.

In [ ]:
#a = df.query('converted == 1').user_id.nunique() 
#b = df['user_id'].nunique()

df.query('converted == 1').user_id.nunique() / df['user_id'].nunique()

e. The number of times the `new_page` and `treatment` don't match.

In [ ]:
## df.query("landing_page == 'new_page' and group == 'treatment' ").count() # match

In [ ]:
##290584 - (len(df.query("landing_page == 'new_page' and group == 'treatment' ")))  # dont match

In [ ]:
## df.groupby(['group', 'landing_page']).count()

In [ ]:
df.query('(group == "treatment" and landing_page != "new_page") or (group != "treatment" and landing_page == "new_page")')['user_id'].count()

f. Do any of the rows have missing values?

In [ ]:
df.isnull().any()
#df.isnull().sum()

`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to figure out how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
df2 = df.drop(df.query('(group == "treatment" and landing_page != "new_page") or (group != "treatment" and landing_page == "new_page") or (group == "control" and landing_page != "old_page") or (group != "control" and landing_page == "old_page")').index)
#df2.head()
df2.shape

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [ ]:
df2['user_id'].nunique()

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
df2[df2.duplicated(['user_id'], keep=False)]['user_id']

c. What is the row information for the repeat **user_id**? 

In [ ]:
df2[df2.user_id.duplicated()]  #User id

#df2[df2['user_id'] == 773192]
#df2[df2.user_id.duplicated()|df2.user_id.duplicated()]

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
df2.drop(axis = 0, index=2893, inplace=True)

In [ ]:
df2[df2.user_id.duplicated()] # checking

#df2[df2['user_id'].duplicated()].count()

`4.` Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
df2['converted'].mean()

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
ctr = df2.query('group=="control"')['converted'].mean()

#(df2.query('converted')['group'] == "control").mean()  

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
trm = df2.query('group=="treatment"')['converted'].mean()

In [ ]:
ctr-trm

d. What is the probability that an individual received the new page?

In [ ]:
df2.query('landing_page == "new_page"').count()[0] /df2.shape[0]

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

**-A direct comparison of proportions of site conversions shows an extremely low difference (0.15%) in favor of the old control page on the new treatment page. Therefore, this result does not show sufficient evidence for more conversions by using the new treatment page.** <br/>
**-A deep analysis is recommended to support wiser decision-making.** 

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

$$H_0: P{old} >= P{new}$$
$$H_1: P{old} < P{new} $$
$$or$$
$$H_0: P{old} - P{new} >= 0$$
$$H_1: P{old} - P{new} < 0$$

**$P(old)$ and $P(new)$ are the old control site and the new treatment site, respectively.**

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

In [ ]:
#success regardling the page
df2['converted'].mean()  

#np.mean(df2['converted'])

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [ ]:
p_new = df2['converted'].mean()
p_new

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [ ]:
p_old = df2['converted'].mean()
p_old

c. What is $n_{new}$, the number of individuals in the treatment group?

In [ ]:
n_new = df2.query('group == "treatment"').shape[0]
n_new
#df2.query('group == "treatment"').count()

d. What is $n_{old}$, the number of individuals in the control group?

In [ ]:
n_old = df2.query('group == "control"').shape[0]
n_old

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
# Binomial distribution
new_page_converted = np.random.binomial(n_new, p_new)

# documentation
# numpy.random.binomial(n, p, size=None)
# n = Parameter of the distribution >= 0
# p = Parameter of the distribution, >= 0 and <=1.
# size = Output shape. (m, n, k)

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
# Binomial distribution
old_page_converted = np.random.binomial(n_old, p_old)

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
obs_diff = (new_page_converted/n_new) - (old_page_converted/n_old) 
obs_diff

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
p_diffs = []
for _ in range(10000):
    old_page_converted = np.random.binomial(n_old,p_old)
    new_page_converted = np.random.binomial(n_new, p_new)
    diff = new_page_converted/n_new - old_page_converted/n_old
    p_diffs.append(diff)    

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diffs);

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
p_diffs = np.array(p_diffs)

In [ ]:
pvalue = (p_diffs > obs_diff).mean() 
pvalue
#p_diffs (new)
#obs_diff (old)

In [ ]:
plt.hist(p_diffs, alpha=.5)
plt.axvline(x=obs_diff, color='red');

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**-This is the p-value.**<br/>
**-With a p-value superior to a type I error rate of 0.05, we have no evidences that the new pages exceeds the old ones in conversions.Therefore, we fail in reject the null hypothesis.**

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = df2.query('group == "control" and converted == 1').shape[0]
convert_new = df2.query('group == "treatment" and converted == 1').shape[0]
n_old = df2[df2['group'] == 'control'].shape[0]
n_new = df2[df2['group'] == 'treatment'].shape[0]
convert_old

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [ ]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')
z_score, p_value   

In [ ]:
from scipy.stats import norm
z_score_critical = (norm.ppf(1-(0.05/2))) # (single-sides test with 95% confidence level)
z_score_critical

# Percent Point Function (PPF): Returns the observation value for the provided probability 
#that is less than or equal to the provided probability from the distribution.
# https://machinelearningmastery.com/critical-values-for-statistical-hypothesis-testing/

In [ ]:
p = norm.cdf(z_score) # confirm with cdf
p

# Cumulative Density Function (CDF): Returns the probability for an 
# observation equal to or lesser than a specific value from the distribution.
# https://machinelearningmastery.com/critical-values-for-statistical-hypothesis-testing/

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**-The z-score is inside our critical value and the p-value is still large.**<br/>
**This means that the z-score and p-value agree with the findings in parts j and k, that is, there is no signifficant difference in the conversions of each landing page, and we cannot reject the null hypothesis.**


<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

    **Logistic Regression. The vaviables are binary.**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2.head()

In [ ]:
df2['intercept']=1
df2[['control','ab_page']] = pd.get_dummies(df2['group'])
df2.drop(labels=['control'], axis=1, inplace=True)
df2.head()

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [ ]:
import statsmodels.api as sm

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
model1 = sm.Logit(df2['converted'], df2[['intercept', 'ab_page']])
result = model1.fit()
result.summary() 

In [ ]:
np.exp(-1.9888), np.exp(-0.0150)

In [ ]:
1/np.exp(-1.9888), 1/np.exp(-0.0150)   #Reciprocal

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  

**-The p-value associated with ab_page is 0.19.**<br/>
<br/>
**-The test in partII is made in one direction (one-tail test) to try to prove that the alternative hypothesis is true (if the new site has more conversions than the old one).**<br/>
**The test in partIII is made in both sides (two-tail test) with a binary categorical dependent variable (conversion or no conversion). Here, the test check if the independent variable (ab_page/new_page) had any effect at all.**<br/>

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**-Aversion to changes can influence user behavior of some people (Simpson's Paradox). Therefore, time (tests duration) is an important variable in tests regression.**<br/>

**-A problem of adding many variables to the model can happen when two or more variables are correlated, when one interfere in another. This situation can cause misunderstanting and bias in testing and interpreting tests.**<br/>

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

In [ ]:
countries_df = pd.read_csv('countries.csv')
df3 = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')
df3.head()

In [ ]:
countries_df['country'].unique()

In [ ]:
df3[['CA','US','UK']] = pd.get_dummies(df3['country'])[['CA','US','UK']]

In [ ]:
model_countries = sm.Logit(df3['converted'],df3[['intercept','US','UK']])
result_countries = model_countries.fit()
result_countries.summary()

In [ ]:
model_countries_2 = sm.Logit(df3['converted'],df3[['intercept','US','UK','ab_page']])
result_countries_2 = model_countries_2.fit()
result_countries_2.summary()

In [ ]:
# considering the type of page, it shows low differences in coef and p-value

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
df3['US_ab_page'] = df3['US'] * df3['ab_page']
df3['UK_ab_page'] = df3['UK'] * df3['ab_page']
df3.head()

In [ ]:
model_countries_3 = sm.Logit(df3['converted'],df3[['intercept','US','UK','ab_page','US_ab_page','UK_ab_page']])
result_countries_3 = model_countries_3.fit()
result_countries_3.summary()

# Conclusions
**-The p-values were demonstrated to be higher than alpha in "Part II" and "Part III" tests, with different methodologies. So, we fail to reject the null hypothesis in both cases.**<br/>
**-Results show no significance in conversions by launching a new page, that suggests the company should keep with the old one.**<br/>
**-Comparing page conversion from different countries, our tests also don't show a significant difference in statistics.It suggests that the variable country does not interfere much in the conversion.**<br/>
**-However, I could suggest the company we run the tests longer, and consider the variable time to discard bias by Simpson's Paradox. Thus avoiding hasty conclusions and financial losses in not launching a new page that  already has been developed and could maybe benefit the company.**


# Sources
- https://www.youtube.com/watch?v=NNpv-n_Hbvc
- https://stackoverflow.com/questions/29337603/scipy-two-tail-ppf-function-for-a-z-value
- https://www.statisticshowto.datasciencecentral.com/
- https://machinelearningmastery.com/critical-values-for-statistical-hypothesis-testing/- 
- Book: An Introduction to Statistical Learning. Gareth James, Daniela Witten, Trevor Hastie, Robert Tibshirani
- http://lindsaymoir.com/wp-content/uploads/2018/05/Analyze_ab_test_results_notebook.html
- https://github.com/IamMohitM/Analyze-A-B-Test-Results/blob/master/Analyze_ab_test_results_notebook.ipynb
- Udacity
- Documentations

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])